In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [2]:
# Program bitstream to FPGA
overlay = Overlay('/home/ubuntu/workspace/design_1.bit')

In [3]:
# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

In [4]:
# Allocate physical memory for AXI DMA MM2S
input_buffer = allocate(shape=(7,), dtype=np.uint64)

In [5]:
# Input
input_buffer[0] = 0x0000B07A057A057A
input_buffer[1] = 0x0000FC6603E10314
input_buffer[2] = 0x0000FC70028F0433
input_buffer[3] = 0xF5A30051FAC21C70
input_buffer[4] = 0x00CC07E10685E399
input_buffer[5] = 0x1400140020002000
input_buffer[6] = 0x1400200014002000

In [6]:
# Check the written data
for i in range(7):
    print("0x%016X" % (input_buffer[i]))

0x0000B07A057A057A
0x0000FC6603E10314
0x0000FC70028F0433
0xF5A30051FAC21C70
0x00CC07E10685E399
0x1400140020002000
0x1400200014002000


In [7]:
# Do AXI DMA MM2S transfer
dma_send.transfer(input_buffer)

In [8]:
# Allocate physical memory for AXI DMA S2MM
output_buffer = allocate(shape=(2,), dtype=np.uint64)

In [9]:
# Check the memory content
for i in range(2):
    print("0x%016X" % (output_buffer[i]))

0x0000000000000000
0x0000000000000000


In [10]:
# Do AXI DMA S2MM transfer
dma_recv.transfer(output_buffer)

In [11]:
# Check the memory content after DMA transfer
for i in range(2):
    print("0x%016X" % (output_buffer[i]))

0x0014002B002B03A8
0x03E803CF03CF004D


In [12]:
out0 = np.zeros(4)
out1 = np.zeros(4)

In [13]:
out0[0] = (output_buffer[0].astype('int64') & 0x000000000000FFFF) / 1024
out1[0] = (output_buffer[1].astype('int64') & 0x000000000000FFFF) / 1024
out0[1] = ((output_buffer[0].astype('int64') & 0x00000000FFFF0000) / pow(2, 16)) / 1024
out1[1] = ((output_buffer[1].astype('int64') & 0x00000000FFFF0000) / pow(2, 16)) / 1024
out0[2] = ((output_buffer[0].astype('int64') & 0x0000FFFF00000000) / pow(2, 32)) / 1024
out1[2] = ((output_buffer[1].astype('int64') & 0x0000FFFF00000000) / pow(2, 32)) / 1024
out0[3] = ((output_buffer[0].astype('uint64') & 0xFFFF000000000000) / pow(2, 48)) / 1024
out1[3] = ((output_buffer[1].astype('uint64') & 0xFFFF000000000000) / pow(2, 48)) / 1024

In [14]:
out0

array([0.9140625 , 0.04199219, 0.04199219, 0.01953125])

In [15]:
out1

array([0.07519531, 0.95214844, 0.95214844, 0.9765625 ])